# Demo: Defining Control_M Workflows using Python

# Step 1 - Setup

## Step 1A - Install the library

In [1]:
# This command needs to be done only once. This installs the library
!pip  install git+https://github.com/tadinve/naga.git

  Cloning https://github.com/tadinve/naga.git to /private/var/folders/jf/ly2qhjns6yn6j46t3rdms2qw0000gp/T/pip-req-build-ijg9qgoo
  Running command git clone -q https://github.com/tadinve/naga.git /private/var/folders/jf/ly2qhjns6yn6j46t3rdms2qw0000gp/T/pip-req-build-ijg9qgoo
  Created wheel for ctm-python-client: filename=ctm_python_client-0.3.0-py3-none-any.whl size=614671 sha256=f2c7e561e6c40899dedd4ad42bbf22e3c5f335141d0f1e209f0024f71918f709
  Stored in directory: /private/var/folders/jf/ly2qhjns6yn6j46t3rdms2qw0000gp/T/pip-ephem-wheel-cache-kak4krqp/wheels/de/35/cb/b70c2ffd6e40942c62635105854bc5fabd5125a0f0f9b17cb9
Successfully built ctm-python-client


In [ ]:
import os, sys
cwd = os.getcwd()
module_path = cwd.split("/")
mp = []
for d in module_path:
    mp.append(d)
    if d == "ctm_python_client":
        break
module_path = "/".join(mp)
module_path


In [ ]:
sys.path.append(module_path)
sys.path

## Step 1B- Import the library

In [2]:
from ctm_python_client.core.bmc_control_m import CmJobFlow
from ctm_python_client.jobs.dummy import DummyJob

# Step 2 - Instantiate, Authenticate and Schedule


## Step 2A - Create the object

In [3]:
# Please change the URfrI, and ctm_user and enter ctm_password to match your environment
from ctm_python_client.session.session import Session

import getpass
ctm_uri =  "https://acb-rhctmv20.centralus.cloudapp.azure.com:8443/automation-api"
ctm_user =  "vtadinad"
ctm_pwd =                                                                                                                                               "P4ssw0rd"
if "ctm_pwd" not in locals(): # has not been enterd once, will skip next time
    ctm_pwd = getpass.getpass("Enter your Control M Password ")

session = Session( endpoint=ctm_uri,username=ctm_user,password=ctm_pwd)
session.get_token()

'34F940FE87612DD1EDF26DCCD589C38479FE312A602F1D00A921544D08BF143928CBD975F24AE109DAD962C6F3701D5F65E56E86E23E6DF8E90A9A77315E39B5'

In [4]:
import graphviz

In [ ]:
t1_flow = CmJobFlow(application="Naga0.2_Demo",sub_application="Demo-02",session=session)

## Step 2B - Define the Schedule

In [ ]:
t1_flow.set_run_as(username="ctmuser",host="acb-rhctmv20")

In [ ]:
# Define the schedule
months = ["JAN", "OCT", "DEC"]
monthDays = ["ALL"]
weekDays = ["MON","TUE", "WED", "THU", "FRI"]
fromTime = "0300"
toTime = "2100"
t1_flow.set_schedule(months, monthDays, weekDays,fromTime, toTime)


# Step 3  - Create Folder

In [ ]:
# Create Fodler
f1 = t1_flow.create_folder(name="HelloWorld")

# Step 4 - Create Tasks

In [ ]:
start = t1_flow.add_job( f1, DummyJob(f1,"Start-Flow") )
end = t1_flow.add_job(f1, DummyJob(f1,"End-Flow") )

hello_world_id = t1_flow.add_job(f1,  DummyJob(f1, "Hello-World" ) )


# Step 5 - Chain Tasks

In [ ]:
# start >>  hello_world_id >> end
t1_flow.chain_jobs(f1,[start, hello_world_id, end])

# Step 6 - Display Workflow

## Step 6A - Display DAG

In [ ]:
# View the t1_flow Details
nodes, edges = t1_flow.get_nodes_and_edges()
nodes, edges

In [ ]:
#display using graphviz
from ctm_python_client.utils.displayDAG import DisplayDAG

DisplayDAG(t1_flow).display_graphviz()

## Step 6B - Display JSON

In [ ]:
t1_flow.display_json()

# Step 7 - Submit Workflow to Control-M

In [ ]:
t1_flow.deploy()

In [ ]:
t1_flow.run()